# Install libs ที่เกี่ยวข้อง

In [4]:
pip install playwright html5lib requests beautifulsoup4 xlrd lxml papermill 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## ดึงข้อมูล diw : กรมอุตสาหกรรม ##

-------------------

# Extract data from diw

In [ ]:
from playwright.async_api import async_playwright
from datetime import datetime
import os

area_skip = []
province_skip = []
district_skip = []

parent_dir = 'diw_2'

async def main():
    start_time = datetime.now()
    broken_excel_file = [] # เพิ่ม list สำหรับเก็บอำเภอที่ดาวน์โหลดไฟล์ไม่ได้
    incomplete_downloads = {} # เพิ่ม dictionary สำหรับเก็บอำเภอที่ดาวน์โหลดไม่ครบ
    
    # กำหนดชื่อไฟล์ล็อก
    log_filename = f"log_diw_{datetime.now().strftime('%Y-%m-%d')}.txt"
    log_path = os.path.join('.', log_filename)
    
    # สร้างโฟลเดอร์สำหรับบันทึกไฟล์
    if not os.path.exists(f"{parent_dir}"):
        os.makedirs(f"{parent_dir}")
        
    async with async_playwright() as playwright:
        # Launch a browser
        browser = await playwright.chromium.launch(headless=False) # headless=False : ให้หน้า browser ขึ้นมาเวลารันเป็นเวลา slow_mo=500 : pop-up ขึ้นมาเป็นเวลา 0.5 secs
        
        # Create a new page
        page = await browser.new_page()

        target_url = 'https://userdb.diw.go.th/factoryPublic/tumbol.asp'

        await page.goto(target_url)
        
        # รอให้หน้าแรกโหลดเสร็จสมบูรณ์ก่อน
        await page.wait_for_load_state('domcontentloaded')

        area_arr = []

        area = await page.locator('//a').all()

        for a in area:
            area_arr.append(await a.text_content())

        # Loop area
        for a in area_arr:
            if a not in area_skip:
                
                print(f"กำลังดำเนินการในภูมิภาค: {a}") # แสดงผลการดำเนินการในระดับภูมิภาค
                await page.locator(f"//a[text()='{a}']").click()
                
                # สร้างโฟลเดอร์ย่อยสำหรับภูมิภาค
                area_dir = f"{parent_dir}/{a}"
                if not os.path.exists(area_dir):
                    os.makedirs(area_dir)
                    
                # รอให้หน้ารายชื่อจังหวัดโหลดเสร็จสมบูรณ์
                await page.wait_for_selector("//a[contains(text(),'จ.')]", timeout=10000)

                province_arr = []

                province = await page.locator("//a[contains(text(),'จ.')]").all()
                
                for p in province:
                    province_arr.append(await p.text_content())
                    
                print(province_arr)

                # Loop province
                for p in province_arr:
                    if p not in province_skip:
                                                
                        print(f"  -> กำลังดำเนินการในจังหวัด: {p}") # แสดงผลการดำเนินการในระดับจังหวัด
                        await page.locator(f"//a[text()='{p}']").click()
                                                
                        # รอให้หน้ารายชื่ออำเภอโหลดเสร็จสมบูรณ์
                        await page.wait_for_load_state('domcontentloaded')

                        # สร้างโฟลเดอร์ย่อยสำหรับจังหวัด
                        province_dir = f"{area_dir}/{p}"
                        if not os.path.exists(province_dir):
                            os.makedirs(province_dir)
                            
                        # รอให้หน้ารายชื่ออำเภอโหลดเสร็จสมบูรณ์
                        await page.wait_for_selector("//a[contains(text(),'อ.')]", timeout=10000)

                        district_arr = []

                        district = await page.locator("//a[contains(text(),'อ.')]").all()

                        for d in district:
                            district_arr.append(await d.text_content())
                            
                        # สร้าง set ของอำเภอที่ต้องดาวน์โหลด
                        expected_districts = {d for d in district_arr if 'กิ่งอำเภอ' not in d and d not in district_skip}

                        # Loop district
                        for d in district_arr:
                            if 'กิ่งอำเภอ' not in d and d not in district_skip:
                                print(f"    -> กำลังดำเนินการในอำเภอ: {d}") # แสดงผลการดำเนินการในระดับอำเภอ
                                await page.locator(f"//a[text()='{d}']").click()
                                
                                # รอให้ปุ่มดาวน์โหลดโหลดเสร็จสมบูรณ์
                                await page.wait_for_selector("//a[contains(text(),'download')]", timeout=10000)

                                try:
                                    # Start waiting for the download
                                    async with page.expect_download() as download_info:
                                        # Perform the action that initiates download
                                        await page.locator(f"//a[contains(text(),'download')]").click()

                                        download = await download_info.value

                                        # Wait for the download process to complete and save the downloaded file somewhere 
                                        
                                        # แก้ไขชื่อไฟล์ให้มีชื่อภูมิภาค, จังหวัด, และอำเภอ
                                        new_filename = f"{p}_{d}_{download.suggested_filename}"
                                        
                                        # บันทึกไฟล์ลงในโฟลเดอร์ที่เหมาะสม
                                        await download.save_as(f"./{parent_dir}/{a}/{p}/{new_filename}")
                                        print(f"      -> บันทึกไฟล์เรียบร้อยแล้ว: {new_filename}")
                                        await page.reload()
                                        
                                except Exception as e:
                                    print(f"เกิดข้อผิดพลาดในการดาวน์โหลดไฟล์สำหรับ {a} / {p} / {d}: {e}")
                                    broken_excel_file.append([a, p, d])
                                    await page.reload()
                                    
                        # ---------------------------------------
                        # ตรวจสอบไฟล์
                        # ---------------------------------------
                        
                        # ดึงรายชื่อไฟล์ที่ดาวน์โหลดมาได้
                        downloaded_files = os.listdir(province_dir)
                        
                        # สร้าง set ของชื่ออำเภอที่ดาวน์โหลดมาได้
                        downloaded_districts = set()
                        for filename in downloaded_files:
                            parts = filename.split('_') # แยกชื่ออำเภอจากชื่อไฟล์ที่กำหนดเอง
                            if len(parts) >= 2:
                                downloaded_districts.add(parts[1])
                                
                        # เปรียบเทียบสอง set เพื่อหาชื่ออำเภอที่ขาดหายไป
                        missing_districts = expected_districts - downloaded_districts
                        if missing_districts:
                            incomplete_downloads[p] = list(missing_districts)
                            print(f" ⚠️ ไฟล์สำหรับจังหวัด {p} ดาวน์โหลดไม่ครบ ขาด: {missing_districts}")
                            
                            # เขียนข้อมูลลงไฟล์ log
                            current_time = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
                            with open(log_path, 'a', encoding='utf-8') as f:
                                for missing_d in missing_districts:
                                    f.write(f"{current_time} : {a}, {p}, {missing_d}\n")
                        else:
                            print(f" ✅ ไฟล์สำหรับจังหวัด {p} ดาวน์โหลดครบทุกอำเภอแล้ว")
            
        await browser.close() 

        end_time = datetime.now()
        diff_time = end_time - start_time
        print(diff_time) 
        print(f"\nการดำเนินการทั้งหมดเสร็จสิ้นแล้ว")
        print(f"ใช้เวลาไปทั้งหมด: {diff_time}")
        print(f"ไฟล์ที่ไม่สามารถดาวน์โหลดได้: {broken_excel_file}")
        print(f"ไฟล์ที่ดาวน์โหลดไม่ครบ: {incomplete_downloads}")

await main()

In [ ]:
# from playwright.async_api import async_playwright
# from datetime import datetime
# import os

# area_skip = []
# province_skip = []
# district_skip = []

# parent_dir = 'diw'

# async def main():
#     start_time = datetime.now()
#     broken_excel_file = [] # เพิ่ม list สำหรับเก็บอำเภอที่ดาวน์โหลดไฟล์ไม่ได้
#     incomplete_downloads = {} # เพิ่ม dictionary สำหรับเก็บอำเภอที่ดาวน์โหลดไม่ครบ
    
#     # กำหนดชื่อไฟล์ล็อก
#     log_filename = f"log_diw_{datetime.now().strftime('%Y-%m-%d')}.txt"
#     log_path = os.path.join('.', log_filename)
    
#     # สร้างโฟลเดอร์สำหรับบันทึกไฟล์
#     if not os.path.exists(f"{parent_dir}"):
#         os.makedirs(f"{parent_dir}")
        
#     async with async_playwright() as playwright:
#         # Launch a browser
#         browser = await playwright.chromium.launch(headless=False) # headless=False : ให้หน้า browser ขึ้นมาเวลารันเป็นเวลา slow_mo=500 : pop-up ขึ้นมาเป็นเวลา 0.5 secs
        
#         # Create a new page
#         page = await browser.new_page()

#         target_url = 'https://userdb.diw.go.th/factoryPublic/tumbol.asp'

#         await page.goto(target_url)
        
#         # รอให้หน้าแรกโหลดเสร็จสมบูรณ์ก่อน
#         await page.wait_for_load_state('domcontentloaded')

#         area_arr = []

#         area = await page.locator('//a').all()

#         for a in area:
#             area_arr.append(await a.text_content())

#         # Loop area
#         for a in area_arr:
#             if a not in area_skip:
                
#                 print(f"กำลังดำเนินการในภูมิภาค: {a}") # แสดงผลการดำเนินการในระดับภูมิภาค
#                 await page.locator(f"//a[text()='{a}']").click()
                
#                 # สร้างโฟลเดอร์ย่อยสำหรับภูมิภาค
#                 area_dir = f"{parent_dir}/{a}"
#                 if not os.path.exists(area_dir):
#                     os.makedirs(area_dir)
                    
#                 # รอให้หน้ารายชื่อจังหวัดโหลดเสร็จสมบูรณ์
#                 await page.wait_for_selector("//a[contains(text(),'จ.')]", timeout=10000)

#                 province_arr = []

#                 province = await page.locator("//a[contains(text(),'จ.')]").all()
                
#                 for p in province:
#                     province_arr.append(await p.text_content())
                    
#                 print(province_arr)

#                 # Loop province
#                 for p in province_arr:
#                     if p not in province_skip:
                                                
#                         print(f"  -> กำลังดำเนินการในจังหวัด: {p}") # แสดงผลการดำเนินการในระดับจังหวัด
#                         await page.locator(f"//a[text()='{p}']").click()
                                                
#                         # รอให้หน้ารายชื่ออำเภอโหลดเสร็จสมบูรณ์
#                         await page.wait_for_load_state('domcontentloaded')

#                         # สร้างโฟลเดอร์ย่อยสำหรับจังหวัด
#                         province_dir = f"{area_dir}/{p}"
#                         if not os.path.exists(province_dir):
#                             os.makedirs(province_dir)
                            
#                         # รอให้หน้ารายชื่ออำเภอโหลดเสร็จสมบูรณ์
#                         await page.wait_for_selector("//a[contains(text(),'อ.')]", timeout=10000)

#                         district_arr = []

#                         district = await page.locator("//a[contains(text(),'อ.')]").all()

#                         for d in district:
#                             district_arr.append(await d.text_content())
                            
#                         # สร้าง set ของอำเภอที่ต้องดาวน์โหลด
#                         expected_districts = {d for d in district_arr if 'กิ่งอำเภอ' not in d and d not in district_skip}

#                         # Loop district
#                         for d in district_arr:
#                             if 'กิ่งอำเภอ' not in d and d not in district_skip:
#                                 print(f"    -> กำลังดำเนินการในอำเภอ: {d}") # แสดงผลการดำเนินการในระดับอำเภอ
#                                 await page.locator(f"//a[text()='{d}']").click()
                                
#                                 # รอให้ปุ่มดาวน์โหลดโหลดเสร็จสมบูรณ์
#                                 await page.wait_for_selector("//a[contains(text(),'download')]", timeout=10000)

#                                 try:
#                                     # Start waiting for the download
#                                     async with page.expect_download() as download_info:
#                                         # Perform the action that initiates download
#                                         await page.locator(f"//a[contains(text(),'download')]").click()

#                                         download = await download_info.value

#                                         # Wait for the download process to complete and save the downloaded file somewhere 
                                        
#                                         # แก้ไขชื่อไฟล์ให้มีชื่อภูมิภาค, จังหวัด, และอำเภอ
#                                         new_filename = f"{p}_{d}_{download.suggested_filename}"
                                        
#                                         # บันทึกไฟล์ลงในโฟลเดอร์ที่เหมาะสม
#                                         await download.save_as(f"./{parent_dir}/{a}/{p}/{new_filename}")
#                                         print(f"      -> บันทึกไฟล์เรียบร้อยแล้ว: {new_filename}")
#                                         await page.reload()
                                        
#                                 except Exception as e:
#                                     print(f"เกิดข้อผิดพลาดในการดาวน์โหลดไฟล์สำหรับ {a} / {p} / {d}: {e}")
#                                     broken_excel_file.append([a, p, d])
#                                     await page.reload()
                                    
#                         # ---------------------------------------
#                         # ตรวจสอบไฟล์
#                         # ---------------------------------------
                        
#                         # ดึงรายชื่อไฟล์ที่ดาวน์โหลดมาได้
#                         downloaded_files = os.listdir(province_dir)
                        
#                         # สร้าง set ของชื่ออำเภอที่ดาวน์โหลดมาได้
#                         downloaded_districts = set()
#                         for filename in downloaded_files:
#                             parts = filename.split('_') # แยกชื่ออำเภอจากชื่อไฟล์ที่กำหนดเอง
#                             if len(parts) >= 2:
#                                 downloaded_districts.add(parts[1])
                                
#                         # เปรียบเทียบสอง set เพื่อหาชื่ออำเภอที่ขาดหายไป
#                         missing_districts = expected_districts - downloaded_districts
#                         if missing_districts:
#                             incomplete_downloads[p] = list(missing_districts)
#                             print(f" ⚠️ ไฟล์สำหรับจังหวัด {p} ดาวน์โหลดไม่ครบ ขาด: {missing_districts}")
                            
#                             # เขียนข้อมูลลงไฟล์ log
#                             current_time = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
#                             with open(log_path, 'a', encoding='utf-8') as f:
#                                 for missing_d in missing_districts:
#                                     f.write(f"{current_time} : {a}, {p}, {missing_d}\n")
#                         else:
#                             print(f" ✅ ไฟล์สำหรับจังหวัด {p} ดาวน์โหลดครบทุกอำเภอแล้ว")
            
#         await browser.close() 

#         end_time = datetime.now()
#         diff_time = end_time - start_time
#         print(diff_time) 
#         print(f"\nการดำเนินการทั้งหมดเสร็จสิ้นแล้ว")
#         print(f"ใช้เวลาไปทั้งหมด: {diff_time}")
#         print(f"ไฟล์ที่ไม่สามารถดาวน์โหลดได้: {broken_excel_file}")
#         print(f"ไฟล์ที่ดาวน์โหลดไม่ครบ: {incomplete_downloads}")

# await main()

กำลังดำเนินการในภูมิภาค: กทม. และภาคกลาง
['จ.กรุงเทพมหานคร', 'จ.สมุทรปราการ', 'จ.นนทบุรี', 'จ.ปทุมธานี', 'จ.พระนครศรีอยุธยา', 'จ.อ่างทอง', 'จ.ลพบุรี', 'จ.สิงห์บุรี', 'จ.ชัยนาท', 'จ.สระบุรี', 'จ.นครนายก', 'จ.นครสวรรค์', 'จ.อุทัยธานี', 'จ.กำแพงเพชร', 'จ.สุโขทัย', 'จ.พิษณุโลก', 'จ.พิจิตร', 'จ.เพชรบูรณ์', 'จ.สุพรรณบุรี', 'จ.นครปฐม', 'จ.สมุทรสาคร', 'จ.สมุทรสงคราม']
  -> กำลังดำเนินการในจังหวัด: จ.กรุงเทพมหานคร
    -> กำลังดำเนินการในอำเภอ: อ.พระนคร
      -> บันทึกไฟล์เรียบร้อยแล้ว: จ.กรุงเทพมหานคร_อ.พระนคร_10-1-.xls
    -> กำลังดำเนินการในอำเภอ: อ.ดุสิต
      -> บันทึกไฟล์เรียบร้อยแล้ว: จ.กรุงเทพมหานคร_อ.ดุสิต_10-2-.xls
    -> กำลังดำเนินการในอำเภอ: อ.หนองจอก
      -> บันทึกไฟล์เรียบร้อยแล้ว: จ.กรุงเทพมหานคร_อ.หนองจอก_10-3-.xls
    -> กำลังดำเนินการในอำเภอ: อ.บางรัก
      -> บันทึกไฟล์เรียบร้อยแล้ว: จ.กรุงเทพมหานคร_อ.บางรัก_10-4-.xls
    -> กำลังดำเนินการในอำเภอ: อ.บางเขน
      -> บันทึกไฟล์เรียบร้อยแล้ว: จ.กรุงเทพมหานคร_อ.บางเขน_10-5-.xls
    -> กำลังดำเนินการในอำเภอ: อ.บางกะปิ
      -> บั

# Append all diw datas

In [7]:
import os
import pandas as pd
from datetime import datetime

start_time = datetime.now()

# Define the root directory to search for files
root_dir = "./diw"
child_dir = 'all'

# Output file path
output_dir = os.path.join(root_dir, child_dir)
output_filename = "diw_client.xlsx"
output_path = os.path.join(output_dir, output_filename)

# List to store dataframes
dfs = []

print("Starting to process Excel files...")

# Use os.walk to traverse the directory tree
for dirpath, dirnames, filenames in os.walk(root_dir):
    # Exclude the output directory from being processed
    if child_dir in dirnames:
        dirnames.remove(child_dir)
    
    # Iterate through all files in the current directory
    for filename in filenames:
        # Check if the file is an Excel file and not the output file itself
        if filename.endswith(".xls") or filename.endswith(".xlsx"):
            # Construct the full file path
            file_path = os.path.join(dirpath, filename)
            
            try:
                # Read the Excel file into a DataFrame
                df = pd.read_excel(file_path)
                
                # Drop the last two rows ที่เป็นเศษทิ้ง เพราะไม่ใช้
                df = df.iloc[:-2, :]
                
                # Split the path and get the folder name that represents the region.
                # The folder list should be ['diw', 'กทม. และภาคกลาง', 'จ.กรุงเทพมหานคร', ...]
                # The region name is at index 1
                path_parts = dirpath.split(os.sep)
                # Ensure the path has at least two parts to avoid an index error
                if len(path_parts) > 1:
                    area_name = path_parts[1]
                else:
                    # Fallback to a default or the full path if the structure is unexpected
                    area_name = "Other"
                
                df['Area'] = area_name
                
                dfs.append(df)
                print(f"  - Processed: {file_path}")
            except Exception as e:
                print(f"  - ERROR processing {file_path}: {e}")

# Check if any data was found before concatenation
if not dfs:
    print("No Excel files were found to process.")
else:
    # Concatenate all dataframes into a single dataframe
    df_concat = pd.concat(dfs, axis=0)

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Save the combined DataFrame to the specified path
    df_concat.to_excel(output_path, index=False)
    print(f"\nSuccessfully combined all data into {output_path}")

end_time = datetime.now()
diff_time = end_time - start_time
print(f"\nTotal processing time: {diff_time}")


Starting to process Excel files...
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.คลองสาน_10-18-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.คลองสามวา_10-46-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.คลองเตย_10-33-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.คันนายาว_10-43-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.จตุจักร_10-30-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.จอมทอง_10-35-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.ดอนเมือง_10-36-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.ดินแดง_10-26-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.ดุสิต_10-2-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.ตลิ่งชัน_10-19-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหา

Final Result Data : https://docs.google.com/spreadsheets/d/11ppgTykt7Db1W0-J-MesBFp5U16PqLvg/edit?gid=1555120110#gid=1555120110

Ref : https://userdb.diw.go.th/factoryPublic/tumbol.asp

In [ ]:
# import os
# import pandas as pd
# from datetime import datetime

# start_time = datetime.now()

# # Define the root directory to search for files
# root_dir = "./diw"
# child_dir = 'all'

# # Output file path
# output_dir = os.path.join(root_dir, child_dir)
# output_filename = "diw_client.xlsx"
# output_path = os.path.join(output_dir, output_filename)

# # List to store dataframes
# dfs = []

# print("Starting to process Excel files...")

# # Use os.walk to traverse the directory tree
# for dirpath, dirnames, filenames in os.walk(root_dir):
#     # Exclude the output directory from being processed
#     if child_dir in dirnames:
#         dirnames.remove(child_dir)
    
#     # Iterate through all files in the current directory
#     for filename in filenames:
#         # Check if the file is an Excel file and not the output file itself
#         if filename.endswith(".xls") or filename.endswith(".xlsx"):
#             # Construct the full file path
#             file_path = os.path.join(dirpath, filename)
            
#             try:
#                 # Read the Excel file into a DataFrame
#                 df = pd.read_excel(file_path)
                
#                 # Drop the last two rows ที่เป็นเศษทิ้ง เพราะไม่ใช้
#                 df = df.iloc[:-2, :]
                
#                 # Split the path and get the folder name that represents the region.
#                 # The folder list should be ['diw', 'กทม. และภาคกลาง', 'จ.กรุงเทพมหานคร', ...]
#                 # The region name is at index 1
#                 path_parts = dirpath.split(os.sep)
#                 # Ensure the path has at least two parts to avoid an index error
#                 if len(path_parts) > 1:
#                     area_name = path_parts[1]
#                 else:
#                     # Fallback to a default or the full path if the structure is unexpected
#                     area_name = "Other"
                
#                 df['Area'] = area_name
                
#                 dfs.append(df)
#                 print(f"  - Processed: {file_path}")
#             except Exception as e:
#                 print(f"  - ERROR processing {file_path}: {e}")

# # Check if any data was found before concatenation
# if not dfs:
#     print("No Excel files were found to process.")
# else:
#     # Concatenate all dataframes into a single dataframe
#     df_concat = pd.concat(dfs, axis=0)

#     # Create the output directory if it doesn't exist
#     os.makedirs(output_dir, exist_ok=True)

#     # Save the combined DataFrame to the specified path
#     df_concat.to_excel(output_path, index=False)
#     print(f"\nSuccessfully combined all data into {output_path}")

# end_time = datetime.now()
# diff_time = end_time - start_time
# print(f"\nTotal processing time: {diff_time}")


Starting to process Excel files...
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.คลองสาน_10-18-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.คลองสามวา_10-46-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.คลองเตย_10-33-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.คันนายาว_10-43-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.จตุจักร_10-30-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.จอมทอง_10-35-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.ดอนเมือง_10-36-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.ดินแดง_10-26-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.ดุสิต_10-2-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.ตลิ่งชัน_10-19-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหา

---

## ดึงงบการเงิน dbd : กรมธุรกิจการค้า ##

----------

# Prepare before extract data

In [ ]:
import pandas as pd

# Import diw data
input_data_path = './diw/all/diw_client.xlsx'
df_template = pd.read_excel(input_data_path)

df = df_template.copy()

# remove ผู้ประกอบการว่าง
df = df.dropna(subset=['ผู้ประกอบการ'])

# Clean data before extract
pattern = r'บริษัท|จำกัด|ห้างหุ้นส่วน|สามัญนิติบุคคล|หนังสือบริคณห์สนธิ|มหาชน|หอการค้า|สมาคมการค้า|จังหวัด|\([^\(\)]*\)' # regex pattern to match unwanted words and text within parentheses 
df['ผู้ประกอบการ'] = df['ผู้ประกอบการ'].str.replace(pattern,'',regex=True).str.strip()

# Sort data before extract
sorter = ['กทม. และภาคกลาง','ภาคตะวันออก','ภาคตะวันตก','ภาคเหนือ','ภาคใต้','ภาคตะวันออกเฉียงเหนือ']

df['Area'] = df['Area'].astype("category") # Convert 'Area' column to categorical type
df['Area'] = df['Area'].cat.reorder_categories(sorter) # Reorder categories based on the defined sorter
df = df.sort_values(['Area']).reset_index(drop=True) # Reset index after sorting
df.to_excel('./diw/diw_client_final_2.xlsx', index=False) # Save the cleaned and sorted DataFrame to a new Excel file
df.head(3)

In [ ]:
# import pandas as pd

# # Import diw data
# input_data_path = './diw/all/diw_client.xlsx'
# df_template = pd.read_excel(input_data_path)

# df = df_template.copy()

# # remove ผู้ประกอบการว่าง
# df = df.dropna(subset=['ผู้ประกอบการ'])

# # Clean data before extract
# pattern = r'บริษัท|จำกัด|ห้างหุ้นส่วน|สามัญนิติบุคคล|หนังสือบริคณห์สนธิ|มหาชน|หอการค้า|สมาคมการค้า|จังหวัด|\([^\(\)]*\)' # regex pattern to match unwanted words and text within parentheses 
# df['ผู้ประกอบการ'] = df['ผู้ประกอบการ'].str.replace(pattern,'',regex=True).str.strip()

# # Sort data before extract
# sorter = ['กทม. และภาคกลาง','ภาคตะวันออก','ภาคตะวันตก','ภาคเหนือ','ภาคใต้','ภาคตะวันออกเฉียงเหนือ']

# df['Area'] = df['Area'].astype("category") # Convert 'Area' column to categorical type
# df['Area'] = df['Area'].cat.reorder_categories(sorter) # Reorder categories based on the defined sorter
# df = df.sort_values(['Area']).reset_index(drop=True) # Reset index after sorting
# df.to_excel('./diw/diw_client_final.xlsx', index=False) # Save the cleaned and sorted DataFrame to a new Excel file
# df.head(3)

,เลขทะเบียนโรงงาน,ชื่อโรงงาน,ผู้ประกอบการ,ประกอบกิจการ,เลขที่,หมู่,ซอย,ถนน,ตำบล,อำเภอ,จังหวัด,ไปรษณีย์,โทรศัพท์,ประเภท,เงินทุน,คนงาน,แรงม้า,TSIC,เลขทะเบียนเดิม,Area
0,50100065625649,ห้างหุ้นส่วนสามัญนิติบุคคล สหประชาพาณิชย์,สหประชาพาณิชย์,พิมพ์สิ่งพิมพ์ต่าง ๆ,377-381,NaN,NaN,ลาดหญ้า,สมเด็จเจ้าพระยา,คลองสาน,กรุงเทพมหานคร,10600,4373994,4101.0,9600000.0,20.0,58.74,17092.0,ก2-41(1)-47/64,กทม. และภาคกลาง
1,20110700125613,บริษัท พิมพ์รุ่ง อินเตอร์ พริ้นท์ จำกัด,พิมพ์รุ่ง อินเตอร์ พริ้นท์,พิมพ์สิ่งพิมพ์ต่างๆ,123,10,แบริ่ง 30/8,สุขุมวิท,สำโรงเหนือ,เมืองสมุทรปราการ,สมุทรปราการ,10270,02-3985380,4101.0,9550000.0,19.0,295.60,17092.0,จ3-41(1)-1/61สป,กทม. และภาคกลาง
2,20110700125407,บริษัท คอมพิวเตอร์เวิลด์ กรุ๊ป จำกัด,คอมพิวเตอร์เวิลด์ กรุ๊ป,พิมพ์สิ่งพิมพ์ต่างๆ และทำสมุด,3000,15,สุขุมวิท 107,สุขุมวิท,สำโรงเหนือ,เมืองสมุทรปราการ,สมุทรปราการ,10270,7498067,4101.0,2100000.0,13.0,94.00,17092.0,จ3-41(1)-1/40สป,กทม. และภาคกลาง


# Extract dbd data

<!-- 
1.ยังทำให้มันรอจบ event แล้วค่อยผ่านไปไม่ได้ 
-->

* demo 
* พิมพ์รุ่ง อินเตอร์ พริ้นท์

In [ ]:
from playwright.async_api import async_playwright
from datetime import datetime
from bs4 import BeautifulSoup
import logging 
import re
import os

# Set up logging configuration
def create_log(log_name,log_output_path):
    logger = logging.getLogger(log_name)
    logger.setLevel(logging.DEBUG)

    filehandler = logging.FileHandler(log_output_path)
    filehandler.setFormatter(
        logging.Formatter("%(asctime)s [%(levelname)s]  %(message)s")
    )
    logger.addHandler(filehandler)

    return logger

# Export file
def export_file(curr_arr,curr_idx,column_name,export_name,export_path):
    curr_arr.append(curr_idx)
    
    dbd_df = pd.DataFrame(curr_arr,columns=column_name)
    
    dbd_df_file = f'{export_name}.csv'
    dbd_df_path = f'{export_path}/{dbd_df_file}'

    dbd_df.to_csv(dbd_df_path)

# Create necessary directories    
def create_directories():
    """
    Creates all necessary directories for log and data files.
    """
    directories = [
        './log',
        './dbd/success',
        './dbd/fail',
        './dbd/data',
        './dbd/error',
        './dbd/no_income_statement',
        './dbd/cannot_get_financial_statement_table',
        './dbd/cannot_access_income_statement'
    ]
    for directory in directories:
        os.makedirs(directory, exist_ok=True)

async def main():
    
    # Create all necessary directories before starting
    create_directories()

    ############### Config ##################
    
    df = pd.read_excel('./diw/diw_client_final.xlsx')
    
    # start_idx = 6610 
    start_idx = 0
    end_idx = len(df)-1

    track_idx_logger_name = 'idx' 
    track_url_logger_name = 'url'
    track_click_logger_name = 'click'
    track_data_logger_name = 'data'

    track_status_logger_name = 'status'

    dbd_success_path = './dbd/success'
    dbd_fail_path = './dbd/fail'

    dbd_data_path = './dbd/data'

    dbd_error_path = './dbd/error'
    dbd_no_income_statement_path = './dbd/no_income_statement'
    dbd_cannot_get_financial_statement_table_path = './dbd/cannot_get_financial_statement_table'
    dbd_cannot_access_income_statement_path = './dbd/cannot_access_income_statement'

    #########################################

    start_time = datetime.now()

    success_idx = []
    fail_idx = []

    data = []

    error_idx = []
    no_income_statement_idx = []
    cannot_get_financial_statement_table_idx = []
    cannot_access_income_statement_idx = []
    
    # column name ปลายทางที่ต้องการเก็บ
    column_name = ['idx',
                   'เลขทะเบียนนิติบุคคล',
                   'เลขทะเบียนโรงงาน',
                   'ชื่อนิติบุคคล',
                   'งบกำไรขาดทุน',
                   'ปี',
                   'จำนวนเงิน',
                   'การเปลี่ยนแปลง',
                   'ประเภทนิติบุคคล',
                   'สถานะนิติบุคคล',
                   'วันที่จดทะเบียนจัดตั้ง',
                   'ทุนจดทะเบียน',
                   'เลขทะเบียนเดิม',
                   'กลุ่มธุรกิจ',
                   'ขนาดธุรกิจ',
                   'ที่ตั้งสำนักงานแห่งใหญ่',
                   'ที่ตั้งตรงกับสำนักงานใหญ่',
                   'หาเจอ',
                    ]
    
    idx_name = ['idx']

    find_again_url = 'https://datawarehouse.dbd.go.th/searchJuristicInfo'

    reg_province = r'(?<=จ\.)\s*[\u0E00-\u0e7f]+|กรุงเทพมหานคร'
    reg_district = r'(?<=อ\.)\s*[\u0E00-\u0e7f]+|(?<=เขต)\s*[\u0E00-\u0e7f]+'
    reg_subdistrict = r'(?<=ต\.)\s*[\u0E00-\u0e7f]+|(?<=แขวง)\s*[\u0E00-\u0e7f]+'

    # Create log
    num_dash = 100

    track_idx_logger = create_log(f'{track_idx_logger_name}',f'./log/{track_idx_logger_name}.log')
    track_url_logger = create_log(f'{track_url_logger_name}',f'./log/{track_url_logger_name}.log')
    track_click_logger = create_log(f'{track_click_logger_name}',f'./log/{track_click_logger_name}.log')
    track_data_logger = create_log(f'{track_data_logger_name}',f'./log/{track_data_logger_name}.log')
    track_status_logger = create_log(f'{track_status_logger_name}',f'./log/{track_status_logger_name}.log')

    async with async_playwright() as playwright:
        # Launch a browser
        browser = await playwright.chromium.launch(headless=False) 

        # Create a new page
        page = await browser.new_page()

        start_url = 'https://datawarehouse.dbd.go.th/index'

        previous_url = start_url
        
        await page.goto(start_url)
        track_url_logger.debug(f'Completed goto {start_url}')

        # กด button ปิด , ยอมรับทั้งหมด
        await page.locator('//button[text()="ปิด"]').click() 
        track_click_logger.debug('Completed Click "ปิด"')
        
        # await page.locator('//button[text()="ยอมรับทั้งหมด"]').click()
        # track_click_logger.debug('Completed Click "ยอมรับทั้งหมด"')

        # Loop ข้อมูลทีละ idx 
        for idx in range(start_idx,end_idx+1):
            broken_web = True
            extract_income_statement = False
            exitLoop = False
            
            # Run again when broken_web
            while broken_web:
                # ถ้า run แล้วเว็บไม่มีปัญหา
                try:
                    track_idx_logger.debug(f'{idx = }')
                    track_url_logger.debug(f'{idx = }')
                    track_click_logger.debug(f'{idx = }')
                    track_data_logger.debug(f'{idx = }')
                    track_status_logger.debug(f'{idx = }')

                    factory_id = df.loc[idx,'เลขทะเบียนโรงงาน']

                    client = df.loc[idx,'ผู้ประกอบการ'].strip()
                    province = df.loc[idx,'จังหวัด'].strip()
                    district = df.loc[idx,'อำเภอ'].strip()
                    subdistrict = df.loc[idx,'ตำบล'].strip()

                    # กรณี search ครั้งแรก
                    if idx==start_idx:
                        input = page.get_by_placeholder('ค้นหาด้วยชื่อหรือเลขทะเบียนนิติบุคคล รหัสประเภทธุรกิจ ชื่อหรือคำอธิบายประเภทธุรกิจ')
                    # กรณี search ครั้งต่อไป
                    else:
                        input = page.get_by_placeholder('ค้นหาด้วยชื่อหรือเลขทะเบียนนิติบุคคล')

                    await input.fill(client)
                    await input.press('Enter') 

                    track_url_logger.debug(f'Completed fill {client = }')

                    current_url = page.url

                    track_url_logger.debug(f'{previous_url = }')
                    track_url_logger.debug(f'{current_url = }')

                    track_status_logger.debug(f'Completed fill {client = } & Press "Enter"')
                    
                    # กรณี search ครั้งเดียวเจอ , url เปลี่ยนตลอด 
                    if current_url != find_again_url and current_url != previous_url:
                        
                        parent = "//*[@id='companyProfileTab1']/div[2]/div[1]/div[1]/div/div/div"

                        corporation_type = await page.locator(f"{parent}/div[text()='ประเภทนิติบุคคล']/following-sibling::div[1]").text_content()

                        corporation_status = await page.locator(f"{parent}/div[text()='สถานะนิติบุคคล']/following-sibling::div[1]").text_content()

                        registration_date = await page.locator(f"{parent}/div[text()='วันที่จดทะเบียนจัดตั้ง']/following-sibling::div[1]").text_content()

                        registered_capital = await page.locator(f"{parent}/div[text()='ทุนจดทะเบียน']/following-sibling::div[1]").text_content()

                        old_corporation_id = await page.locator(f"{parent}/div[text()='เลขทะเบียนเดิม']/following-sibling::div[1]").text_content()

                        business_type = await page.locator(f"{parent}/div[text()='กลุ่มธุรกิจ']/following-sibling::div[1]").text_content()

                        business_size = await page.locator(f"{parent}/div[text()='ขนาดธุรกิจ']/following-sibling::div[1]").text_content()

                        center_location = await page.locator(f"{parent}/div[text()='ที่ตั้งสำนักงานแห่งใหญ่']/following-sibling::div[1]").text_content()

                        found = 'Yes'

                        # check ที่ตั้งสำนักงานใหญ่ 
                        if province == re.findall(reg_province,center_location)[0].strip() and \
                            district == re.findall(reg_district,center_location)[0].strip() and \
                            subdistrict == re.findall(reg_subdistrict,center_location)[0].strip():
                            same_center_location = 'Yes'

                        else:
                            same_center_location = 'No'

                        await page.locator('//span[text()="ข้อมูลงบการเงิน"]').click()
                        track_click_logger.debug('Completed Click "ข้อมูลงบการเงิน"')

                        # Define Boolean broken_web
                        broken_web = False

                        # check ว่ามีข้อมูลงบกำไรขาดทุนไหม ถ้ามีดึงมันมา 
                        try:
                            await page.locator('//span[text()="งบกำไรขาดทุน"]').click()
                            track_click_logger.debug('Completed Click "งบกำไรขาดทุน"')
    
                            track_status_logger.debug(f'Has income statement')

                            # check ว่าสามารถดึงงบกำไรขาดทุนมาได้ไหม ถ้าได้ดึงมาปกติ ถ้าไม่ได้ให้ดึงอีกครั้ง
                            while not extract_income_statement:
                                try:
                                    # locate ไปที่ income statement table 
                                    table_loc = page.locator("(//div[@class='table-responsive'])[1]")
                                    table = BeautifulSoup(await table_loc.inner_html(), 'html.parser')
                                    track_data_logger.debug(f'Completed Access table')

                                    # current year - 1 นับถอยหลังไป 5 
                                    year_loc = page.locator("//table[@class='table table-hover text-end table-fixed']/thead/tr[1]")
                                    year = BeautifulSoup(await year_loc.inner_html(), 'html.parser')
                                    track_data_logger.debug(f'Completed Access year')
                                
                                    years = []

                                    for y in year.find_all('th'):
                                        years.append(y.text)

                                    years = years[1:]

                                    corporation_id = await page.locator("//h4[contains(text(),'เลขทะเบียนนิติบุคคล')]").text_content()
                                    corporation_id = corporation_id.split(':')[1]

                                    corporation_name = await page.locator("//h3[contains(text(),'ชื่อนิติบุคคล')]").text_content()
                                    corporation_name = corporation_name.split(':')[1]

                                    track_status_logger.debug(f'can_get_financial_statement_table')

                                    for body in table.find_all('tbody'):
                                        for rows in body.find_all('tr'):
                                            for income_states in rows.find_all('th'):
                                                # check ว่าสามารถดึงงบกำไรขาดทุนมาได้ไหม ถ้าได้ดึงมาปกติ ถ้าไม่ได้ให้ดึงอีกครั้ง
                                                if income_states.text == 'รายได้หลัก':
                                                    # Define Boolean extract_income_statement
                                                    extract_income_statement = True
                                                
                                                if extract_income_statement:
                                                    for i,year_state in enumerate(years):
                                                        values = rows.find_all('td')[2*i:2*(i+1)]
                                                        amount = values[0].text
                                                        change_amount = values[1].text

                                                        data.append([idx,
                                                                    corporation_id,
                                                                    factory_id,
                                                                    corporation_name,
                                                                    income_states.text,
                                                                    year_state,
                                                                    amount,
                                                                    change_amount,
                                                                    corporation_type,
                                                                    corporation_status,
                                                                    registration_date,
                                                                    registered_capital,
                                                                    old_corporation_id,
                                                                    business_type,
                                                                    business_size,
                                                                    center_location,
                                                                    same_center_location,
                                                                    found])

                                                else:
                                                    exitLoop = True
                                                    break
                                            if exitLoop:
                                                break
                                        if exitLoop:
                                            break

                                    if extract_income_statement:
                                        track_data_logger.debug(f'Completed Extract Income Statement')
                                        
                                        # Export success_idx
                                        export_file(success_idx,
                                                    idx,
                                                    idx_name,
                                                    f'success_idx {start_idx}',
                                                    dbd_success_path)
                                        
                                        track_status_logger.debug(f'can_access_income_statement')

                                    else:
                                        # Export cannot_access_income_statement_idx
                                        export_file(cannot_access_income_statement_idx,
                                                idx,
                                                idx_name,
                                                f'cannot_access_income_statement_idx {start_idx}'
                                                ,dbd_cannot_access_income_statement_path)
                                        
                                        # Export fail_idx
                                        export_file(fail_idx,
                                                    idx,
                                                    idx_name,
                                                    f'fail_idx {start_idx}'
                                                    ,dbd_fail_path)
                                        
                                        track_status_logger.error(f'cannot_access_income_statement')

                                except:
                                    # Export cannot_get_financial_statement_table_idx
                                    export_file(cannot_get_financial_statement_table_idx,
                                                idx,
                                                idx_name,
                                                f'cannot_get_financial_statement_table_idx {start_idx}'
                                                ,dbd_cannot_get_financial_statement_table_path)

                                    # Export fail_idx
                                    export_file(fail_idx,
                                                idx,
                                                idx_name,
                                                f'fail_idx {start_idx}'
                                                ,dbd_fail_path)
                                    
                                    track_status_logger.error(f'cannot_get_financial_statement_table')

                        except:
                            # Export no_income_statement_idx
                            export_file(no_income_statement_idx,
                                        idx,
                                        idx_name,
                                        f'no_income_statement_idx {start_idx}'
                                        ,dbd_no_income_statement_path)

                            # Export success_idx
                            export_file(success_idx,
                                        idx,
                                        idx_name,
                                        f'success_idx {start_idx}',
                                        dbd_success_path)
                            
                            data.append([idx,
                                        '',
                                        factory_id,
                                        '',
                                        '',
                                        '',
                                        '',
                                        '',
                                        corporation_type,
                                        corporation_status,
                                        registration_date,
                                        registered_capital,
                                        old_corporation_id,
                                        business_type,
                                        business_size,
                                        center_location,
                                        same_center_location,
                                        found])
                            
                            track_status_logger.error(f'No income statement')
                            
                    # กรณี search ครั้งเดียวไม่เจอ , url จะไม่เปลี่ยน 
                    else:
                        data.append([idx]+['']+[factory_id]+['']*(len(column_name)-3)+['No'])

                        # Export success_idx
                        export_file(success_idx,
                                    idx,
                                    idx_name,
                                    f'success_idx {start_idx}',
                                    dbd_success_path)

                    previous_url = current_url

                    # Export dbd data (Error this definitely)
                    dbd_df = pd.DataFrame(data,columns=column_name)
                    
                    dbd_df_file = f'dbd_client {start_idx}.csv'
                    dbd_df_path = f'{dbd_data_path}/{dbd_df_file}'

                    dbd_df.to_csv(dbd_df_path)

                    if extract_income_statement:
                        track_status_logger.info('Success') 
                    else:
                        track_status_logger.error('Fail') 
                    
                    track_idx_logger.debug('-'*num_dash)
                    track_url_logger.debug('-'*num_dash)
                    track_click_logger.debug('-'*num_dash)
                    track_data_logger.debug('-'*num_dash)
                    track_status_logger.debug('-'*num_dash)

                # ถ้า run แล้วเว็บมีปัญหา
                except:                    
                    # Export error_idx
                    export_file(error_idx,
                                idx,
                                idx_name,
                                f'error_idx {start_idx}',
                                dbd_error_path)        

                    # Export fail_idx
                    export_file(fail_idx,
                                idx,
                                idx_name,
                                f'fail_idx {start_idx}',
                                dbd_fail_path)
                    
                    track_status_logger.error('Fail') 
            
                    track_idx_logger.debug('-'*num_dash)
                    track_url_logger.debug('-'*num_dash)
                    track_click_logger.debug('-'*num_dash)
                    track_data_logger.debug('-'*num_dash)
                    track_status_logger.debug('-'*num_dash)

        await browser.close() 
            
        end_time = datetime.now()
        diff_time = end_time - start_time
        print(diff_time)

if __name__ == "__main__":
    await main()    

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


---

# Clean data